In [0]:
from google.colab import drive
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys, random, os

drive.mount('/content/drive')
os.chdir("/content/drive/My Drive/capstone_code")
sys.path.append("/content/drive/My Drive/capstone_code/p_file")

train = pd.read_csv("./training_set.csv")
test = pd.read_csv("./test_set.csv")

In [0]:
def load_data(nframe=5, n_aug=0, skip_frame=1):
    train = pd.read_csv("./training_set.csv")
    test = pd.read_csv("./test_set.csv")
    from preprocessing.serialization import serialize_data_clf, serialize_data_clf_val
    from models.data_generator import data_generator_clf

    train_input, train_label, encoder =  serialize_data_clf(nframe, train, n_aug=n_aug, skip_frame=skip_frame, save=True, save_path="./")
    val_input, val_label, encoder =  serialize_data_clf_val(encoder, nframe, test, n_aug=n_aug, skip_frame=skip_frame, save=True, save_path="./")

    train_generator = data_generator_clf(train_input, train_label, batch_size=64)
    val_generator = data_generator_clf(val_input, val_label, batch_size=64, augmented=False)  
    return train_generator, val_generator, encoder

def train_model_org(save_path, nframe=5, n_aug=0, skip_frame=1, n_epoch=100, n_latent_layer = 3, latent_unit = 100, dropout_rate = 0.2, lr = 0.001, save_train_stats=True):
      from models.classification import classification_model
      model = classification_model(save_dir = save_path, nframe=nframe, n_latent_layer = n_latent_layer, latent_unit = latent_unit, dropout_rate = dropout_rate, lr = lr, early_stop=False)
      train_generator, val_generator, encoder = load_data(nframe, n_aug, skip_frame)
      print("Training model:"+save_path)
      history = model.training_generator(train_generator, val_generator, 500, 200, n_epoch=n_epoch)
      if save_train_stats: 
          np.array(history.history['acc']).dump(save_path+"_stats_tacc")
          np.array(history.history['val_acc']).dump(save_path+"_stats_vacc")
          np.array(history.history['loss']).dump(save_path+"_stats_tlosss")
          np.array(history.history['val_loss']).dump(save_path+"_stats_vloss")
      return model, history, encoder

In [0]:
nframes = [1, 5, 10]
skip_frames = [1, 2, 4]

for nframe in nframes:
    for skip_frame in skip_frames: 
        save_path = "clf_{}frame_{}skip_default".format(nframe, skip_frame)
        train_model_org(save_path, nframe=nframe, skip_frame=skip_frame, save_train_stats=True)

In [0]:
n_latent_layers = [3, 6, 9]
latent_units = [100, 200, 300]


for n_latent_layer in n_latent_layers:
    for latent_unit in latent_units: 
        save_path = "clf_5frame_2skip_{}layer_{}unit".format(n_latent_layer, latent_unit)
        train_model_org(save_path, nframe=5, skip_frame=2, n_latent_layer = n_latent_layer, latent_unit = latent_unit, save_train_stats=True)

In [0]:
dropout_rates = [0.1, 0.2, 0.25]
lrs = [0.001, 0.05, 0.01]

for dropout_rate in dropout_rates:
    for lr in lrs: 
        save_path = "clf_5frame_2skip_{}dr_{}lr".format(dropout_rate, lr)
        train_model_org(save_path, nframe=5, skip_frame=2, dropout_rate = dropout_rate, lr = lr, save_train_stats=True)